In [2]:
import os
import os.path as osp

import numpy as np
import torch
import torch.nn as nn
from torchvision import models
import torchvision.transforms as transforms
from PIL import Image
import pickle as pkl

In [4]:
im_path = 'data/middlebury/images/'
out_path = 'data/middlebury/features/'

In [35]:
features_image1 = out_path + 'Backyard/frame07.pkl'
features_image2 = out_path + 'Backyard/frame08.pkl'
image1 = im_path + 'Backyard/frame07.png'
image2 = im_path + 'Backyard/frame08.png'

features1 = pkl.load( open(features_image1, "rb" ) )
features2 = pkl.load( open(features_image2, "rb" ) )

tensor_F1 = torch.from_numpy(features1["F"]).t()
tensor_U1 = torch.from_numpy(features1["U"]).t()
tensor_F2 = torch.from_numpy(features2["F"]).t()
tensor_U2 = torch.from_numpy(features2["U"]).t()


In [36]:
# TESTING -------------
M_p = torch.mm(tensor_U1, tensor_U2.t())
M_p.view(M_p.numel()).shape[0] == M_p.shape[0] * M_p.shape[1]

print(tensor_F1.shape)
# TESTING -------------

torch.Size([100, 1536])


In [95]:
def kronecker(matrix1, matrix2):
    return torch.ger(matrix1.view(-1), matrix2.view(-1)).reshape(*(matrix1.size() + matrix2.size())).permute([0, 2, 1, 3]).reshape(matrix1.size(0) * matrix2.size(0), matrix1.size(1) * matrix2.size(1))


def buildGraphStructure(A):
    """
    Arguments:
    ----------
        - A: node-to-node adjaceny matrix
        
    Returns:
    --------
        - G and H: node-edge incidence matrices such that: A = G*H^T
    
    """
    
    # Get number of nodes
    n = A.shape[0]
    
    # Count number of ones in the adj. matrix to get number of edges
    nr_edges = torch.sum(A)
    
    # Init G and H
    G = torch.zeros(n, nr_edges) 
    H = torch.zeros(n, nr_edges)
    
    # Get all non-zero entries and build G and H
    entries = (A != 0).nonzero()
    for count, (i,j) in enumerate(entries, start=0):
        G[i, count] = 1
        H[j, count] = 1
        
    return [G, H]


def affinityMatrix_forward(F1, F2, U1, U2, G1, G2, H1, H2):
    """
    Arguments:
    ----------
        - F1, F2: edge features of input image 1 and 2
        - U1, U2: node features of input image 1 and 2
        - G1, H1: node-edge incidence matrices of image 1
        - G2, H2: node-edge incidence matrices of image 2
    
    Returns:
    ----------
        - M: affinity Matrix computed according to the given formula
    
    """
    
    # => TODO: lambda init for trivial test
    lam = torch.ones(F1.shape[0], F1.shape[0])
    
    # (a) Build X and Y
    #     - Get ordering of edges from G and H
    #     - Extract edge features of start and end nodes and concat
    idx1_start = (G1 != 0).nonzero()[:,0]
    idx2_start = (G2 != 0).nonzero()[:,0]
    idx1_end = (H1 != 0).nonzero()[:,0]
    idx2_end = (H2 != 0).nonzero()[:,0]
    X = torch.cat((F1[np.array(idx1_start)], F1[np.array(idx1_end)]), 1)
    Y = torch.cat((F2[np.array(idx2_start)], F2[np.array(idx2_end)]), 1)
    
    # (b) Calculate M_e = X * \lambda * Y^T
    M_e = torch.mm(torch.mm(X, lam), Y)
    
    # (c) Calculate M_p = U1 * U2^T
    M_p = torch.mm(U1, U2.t())
    
    # Calculate M = [vec(M_p)] + (G_2 \kronecker G_1)[vec(M_e)](H_2 \kronecker H_1)^T
    diagM_p = torchl.diag(M_p.view(M_p.numel()))
    diagM_e = torchl.diag(M_e.view(M_e.numel()))
    M = diagM_p + torch.mm(torch.mm(kronecker(G2, G1), diagM_e), kronecker(H2, H1).t())
    
    return M

def powerIteration_forward(M, N):
    """
    Arguments:
    ----------
        - M: affinity matrix

    Returns:
    --------
        - v*: optimal assignment vector (computed using power iterations) 
    """
    
    # Init starting v
    v = torch.ones(M.shape[1],1)
    
    # Perform N iterations: v_k+1 = M*v_k / (||M*v_k||_2) 
    for i in range(N):
        v = torch.mv(M, v)
        v = v / torch.norm(v, 2)
        
    return tensor_v    
        
    
def biStochastic_forward(v, n, m):
    """
    Arguments:
    ----------
        - v:     optimal assignment vector
        - n, m:  dimension of nodes of image 1 and image 2
        
    Returns:
    --------
        - S:    double stochastic confidence matrix S
    
    """
    
    # Reshape the assignment vector to matrix form
    S = v.view(n,m)
    
    # Perform N iterations: S_k+1 = ...., S_k+2 = ...
    for i in range(N):
        S = torch.mm(S, torch.mm(torch.ones(1,n),S).inverse())
        S = torch.mm(torch.mv(S, torch.ones(m,1)).inverse(), S)
        
    return S
    

In [71]:
# Some tests
# -----------
test = torch.from_numpy(np.asarray([[1,0,1],[1,1,1],[1,0,0]]))
[G, H] = buildGraphStructure(test)
print(G)
print(H)

tensor([[1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1.]])
tensor([[1., 0., 1., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 1., 0.]])
